In [27]:
!python -m pip install ebooklib BeautifulSoup4 spacy textblob openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.5.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (3.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
Using cached jiter-0.5.0-cp311-cp311-macosx_11_0_arm64.whl (299 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [24]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.8 MB/s eta 0:00:00MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [18]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import re
import os

# Load the EPUB file
book = epub.read_epub('data/1984.epub')

# Create a directory to store the output chapters
output_dir = 'chapters'
os.makedirs(output_dir, exist_ok=True)

# Initialize variables for storing the entire book content
full_text = ""

# Extract the full text from the EPUB file
for item in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
    content = item.get_body_content()
    
    if content:
        soup = BeautifulSoup(content, 'html.parser')
        text = soup.get_text(separator=' ', strip=True)
        full_text += text + "\n"  # Add a newline between items for separation

# Split the text into parts and chapters
# Looking for "Part One", "Part Two" and "Chapter X" patterns
parts_and_chapters = re.split(r'(Part [A-Z][a-z]+|PART [A-Z][a-z]+|Chapter [0-9]+|CHAPTER [0-9]+)', full_text)

# Initialize variables for storing the current part and chapters
current_part = None
chapter_data = []

# Combine part titles, chapter titles, and their content
for i in range(1, len(parts_and_chapters), 2):
    section_title = parts_and_chapters[i].strip()
    section_content = parts_and_chapters[i+1].strip()
    
    # If the section is a "Part", update current_part and continue
    if re.match(r'Part [A-Z][a-z]+|PART [A-Z][a-z]+', section_title):
        current_part = section_title  # Track the current part
    else:
        # If it's a chapter, save the part and chapter content
        chapter_data.append((current_part, section_title, section_content))

# Store each part and chapter in a separate text file
for idx, (part, chapter_title, content) in enumerate(chapter_data):
    # Create a file name that includes both the part and chapter number
    part_suffix = part.replace(" ", "_") if part else "No_Part"
    filename = os.path.join(output_dir, f"{part_suffix}_Chapter_{idx+1}.txt")
    
    # Write the chapter content to the file
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(f"{part}\n{chapter_title}\n\n{content}")
    
    print(f"Saved {chapter_title} in {filename}")

print("Chapters saved successfully.")


Saved Chapter 1 in chapters/Part_One_Chapter_1.txt
Saved Chapter 2 in chapters/Part_One_Chapter_2.txt
Saved Chapter 3 in chapters/Part_One_Chapter_3.txt
Saved Chapter 4 in chapters/Part_One_Chapter_4.txt
Saved Chapter 5 in chapters/Part_One_Chapter_5.txt
Saved Chapter 6 in chapters/Part_One_Chapter_6.txt
Saved Chapter 7 in chapters/Part_One_Chapter_7.txt
Saved Chapter 8 in chapters/Part_One_Chapter_8.txt
Saved Chapter 1 in chapters/Part_Two_Chapter_9.txt
Saved Chapter 2 in chapters/Part_Two_Chapter_10.txt
Saved Chapter 3 in chapters/Part_Two_Chapter_11.txt
Saved Chapter 4 in chapters/Part_Two_Chapter_12.txt
Saved Chapter 5 in chapters/Part_Two_Chapter_13.txt
Saved Chapter 6 in chapters/Part_Two_Chapter_14.txt
Saved Chapter 7 in chapters/Part_Two_Chapter_15.txt
Saved Chapter 8 in chapters/Part_Two_Chapter_16.txt
Saved Chapter 9 in chapters/Part_Two_Chapter_17.txt
Saved Chapter 1 in chapters/Part_Three_Chapter_18.txt
Saved Chapter 2 in chapters/Part_Three_Chapter_19.txt
Saved Chapter 3 i

In [26]:
import spacy
from textblob import TextBlob

# Load the English language model for spaCy
nlp = spacy.load("en_core_web_sm")

# Function to detect key points and sentiment
def analyze_chapter_for_reflection(content):
    # Process the content with spaCy for key sentences
    doc = nlp(content)
    
    key_points = []
    sentiment_changes = []
    
    # Iterate over the sentences
    for sent in doc.sents:
        # Check the length of the sentence to filter key points (longer sentences might carry more weight)
        if len(sent.text.split()) > 10:  # Threshold can be adjusted
            key_points.append(sent.text)
        
        # Use TextBlob to assess sentiment of the sentence
        sentiment = TextBlob(sent.text).sentiment.polarity
        
        # Capture major sentiment changes
        if abs(sentiment) > 0.3:  # Threshold for emotional intensity
            sentiment_changes.append((sent.text, sentiment))
    
    return key_points, sentiment_changes

Key Points:
Part One
Chapter 1

I t was a bright cold day in April, and the clocks were striking thirteen.
Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.
At one end of it a coloured poster, too large for indoor display, had been tacked to the wall.
It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features.
Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours.
It was part of the economy drive in preparation for Hate Week.
The flat was seven flights up, and Winston, who was thirty-nine and had a varicose ulcer above his right ankle, went slowly, resting several times on the way.
On each landing, opposite the lift-shaft, the poster with 

In [38]:
import openai

# Function to generate reflection prompts using GPT-4
def generate_reflection_prompt(key_point):
    completion = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Generate a reflective question based on this sentence: '{key_point}'"}
        ]
    )
    
    # Extract the generated prompt
    prompt = completion.choices[0].message.content
    return prompt


In [34]:
# Analyze a chapter and generate reflection prompts
def analyze_and_generate_reflection_prompts(chapter_content):
    key_points, sentiment_changes = analyze_chapter_for_reflection(chapter_content)
    
    # Generate reflection prompts for each key point
    reflection_prompts = [generate_reflection_prompt(point) for point in key_points]
    
    return key_points, reflection_prompts

In [39]:
import json

# Example of processing a chapter
with open("chapters/Part_One_Chapter_1.txt", 'r', encoding='utf-8') as file:
    chapter_content = file.read()

# Analyze the content and generate reflection prompts
key_points, reflection_prompts = analyze_and_generate_reflection_prompts(chapter_content)

# Print the results
print("Key Points:")
for point in key_points:
    print(point)

print("\nGenerated Reflection Prompts:")
for prompt in reflection_prompts:
    print(prompt)

# Store the reflection prompts as metadata in JSON
reflection_metadata = {
    "chapter_1": {
        "reflection_points": [
            {
                "key_sentence": key_points[i],
                "reflection_prompt": reflection_prompts[i],
                "pause_duration": 15  # Default pause duration
            } for i in range(len(key_points))
        ]
    }
}

# Save the metadata to a JSON file
with open('reflection_metadata.json', 'w', encoding='utf-8') as f:
    json.dump(reflection_metadata, f, indent=4)

print("Reflection prompts saved to reflection_metadata.json.")

Key Points:
Part One
Chapter 1

I t was a bright cold day in April, and the clocks were striking thirteen.
Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.
At one end of it a coloured poster, too large for indoor display, had been tacked to the wall.
It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features.
Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours.
It was part of the economy drive in preparation for Hate Week.
The flat was seven flights up, and Winston, who was thirty-nine and had a varicose ulcer above his right ankle, went slowly, resting several times on the way.
On each landing, opposite the lift-shaft, the poster with 